In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from scipy import ndimage
from func import *

In [2]:
####################### AGN Spectrum Modelling ##########################################
def scale_factor_spectrum(z,data,error,wo_wave):
    k = 1+z
    select1 = (wo_wave>6750*k) & (wo_wave<6800*k)
    cont_dat = data[select1]
    cont_err = error[select1]
    error_maincube = np.std(cont_dat)
    error_errcube = np.mean(cont_err)
    fact = error_maincube/error_errcube
    return fact

def popt_central_fit(wo_wave,data,error,obj,p_init,z,MC_loops,broad2=False):
    k = 1+z
    select = (wo_wave>4750*k) & (wo_wave<5090*k)
    if broad2:
        full_gauss = full_gauss2
    else:
        full_gauss = full_gauss1
    popt_full_fit,pcov_full_fit = leastsq(full_gauss,x0=p_init,args=(wo_wave[select],data[select],error[select]),maxfev = 10000000)
    parameters_MC = np.zeros((len(popt_full_fit),MC_loops))
    for l in range(MC_loops):
        iteration_data = np.random.normal(data[select],error[select]) 
        popt_MC,pcov_MC =leastsq(full_gauss,x0=p_init,args=(wo_wave[select],iteration_data,error[select]),maxfev = 10000000)
        parameters_MC[:,l]=popt_MC
    parameters_err = np.std(parameters_MC,1)  
    (amp_Hb_error,amp_OIII5007_error,vel_OIII_error,vel_sigma_OIII_error,amp_Hb_br_error,amp_OIII5007_br_error,vel_OIII_br_error,vel_sigma_OIII_br_error,amp_Hb1_error,amp_Fe5018_1_error,vel_Hb1_error,vel_sigma_Hb1_error,amp_Hb2_error,amp_Fe5018_2_error,vel_Hb2_error,vel_sigma_Hb2_error,m_error,c_error) = parameters_err
    return popt_full_fit,parameters_err 

In [3]:
################### Creating 9" aperture cube #################################
def create_9_arcsec_minicube(wo_cube,wo_err,wo_header,cont_cube,brightest_pixel_x,brightest_pixel_y,muse_sampling_size,box_size):
    if muse_sampling_size == 0.4:
        mini_cube_data = wo_cube[:,brightest_pixel_y-box_size:brightest_pixel_y+box_size+1,brightest_pixel_x-box_size:brightest_pixel_x+box_size+1]
        mini_cube_err = wo_err[:,brightest_pixel_y-box_size:brightest_pixel_y+box_size+1,brightest_pixel_x-box_size:brightest_pixel_x+box_size+1]
    else:
        mini_cube_data = wo_cube[:,brightest_pixel_y-2*box_size:brightest_pixel_y+2*box_size+1,brightest_pixel_x-2*box_size:brightest_pixel_x+2*box_size+1]
        mini_cube_err = wo_err[:,brightest_pixel_y-2*box_size:brightest_pixel_y+2*box_size+1,brightest_pixel_x-2*box_size:brightest_pixel_x+2*box_size+1]
    wo_header['CRPIX1'] = wo_header['CRPIX1'] - (brightest_pixel_x-box_size)
    wo_header['CRPIX2'] = wo_header['CRPIX2'] - (brightest_pixel_y-box_size)
    return mini_cube_data, mini_cube_err,wo_header

def emp_fact(z,mini_cube,mini_err,wo_wave):
    true_err = np.zeros((np.shape(wo_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    for i in range(mini_cube.shape[1]):
        for j in range(mini_cube.shape[2]):
            k = 1+z
            select1 = (wo_wave>6750*k) & (wo_wave<6850*k)
   
            (spectrum,error) = (mini_cube[:,i,j],mini_err[:,i,j])  
            (spec,err) = (spectrum[select1],error[select1])
            (error_maincube,error_errcube) = (np.std(spec),np.mean(err))
            fact = error_maincube/error_errcube
            new_err = fact*error
            true_err[:,i,j] = new_err 
    return true_err
    

In [4]:
#################### Creating 9" aperture flux map ########################################
def spectroastrometry(mini_cube,mini_err,mini_wave,fixed_param,MC_loops,broad2=False):
    output_par = np.zeros((10,mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    output_par_err = np.zeros((10,mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    fitted = np.zeros((np.shape(mini_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    residual = np.zeros((np.shape(mini_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    if broad2:
        full_gauss_fixkin = full_gauss2_fixkin
    else:
        full_gauss_fixkin = full_gauss1_fixkin
    for i in range(mini_cube.shape[1]):
        for j in range(mini_cube.shape[2]):
            (spectrum,error) = (mini_cube[:,i,j],mini_err[:,i,j])    
            popt_full_fit,pcov_full_fit = leastsq(full_gauss_fixkin,x0=[0.1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, -0.7, 0.001],args=(mini_wave,spectrum,error,fixed_param),maxfev = 10000000)
            
            output_par[:,i,j]=popt_full_fit
            fitted[:,i,j]=(full_gauss_fixkin(popt_full_fit,mini_wave,spectrum,error,fixed_param))*(error)+spectrum 
            residual[:,i,j] = mini_cube[:,i,j] - fitted[:,i,j]
            
            spec_parameters_MC = np.zeros((len(popt_full_fit),MC_loops))
            for l in range(MC_loops):
                iteration_data = np.random.normal(spectrum,error)   
                popt_spec_MC,pcov_spec_MC = leastsq(full_gauss_fixkin,x0=[0.1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, -0.7, 0.001],args=(mini_wave,iteration_data,error,fixed_param),maxfev = 10000000)
                spec_parameters_MC[:,l]=popt_spec_MC
                
            spec_parameters_err = np.std(spec_parameters_MC,1)
            output_par_err[:,i,j] = spec_parameters_err
    return output_par,output_par_err,fitted,residual

def flux_map(output_par,obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(output_par[0,:,:],name='amp_Hb'))
    hdus.append(fits.ImageHDU(output_par[1,:,:],name='amp_OIII5007'))
    hdus.append(fits.ImageHDU(output_par[2,:,:],name='amp_OIII5007_br'))
    hdus.append(fits.ImageHDU(output_par[3,:,:],name='amp_Hb_br'))
    hdus.append(fits.ImageHDU(output_par[4,:,:],name='amp_Hb1'))
    hdus.append(fits.ImageHDU(output_par[5,:,:],name='amp_Hb2'))
    hdus.append(fits.ImageHDU(output_par[6,:,:],name='amp_Fe5018_1'))
    hdus.append(fits.ImageHDU(output_par[7,:,:],name='amp_Fe5018_2'))
    hdus.append(fits.ImageHDU(output_par[8,:,:],name='m'))
    hdus.append(fits.ImageHDU(output_par[9,:,:],name='c'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/9_arcsec_subcube_par_%s_MC.fits'%(destination_path_cube,obj,obj),overwrite='True')

def flux_map_err(output_par_err,obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(output_par_err[0,:,:],name='amp_Hb_err'))
    hdus.append(fits.ImageHDU(output_par_err[1,:,:],name='amp_OIII5007_err'))
    hdus.append(fits.ImageHDU(output_par_err[2,:,:],name='amp_OIII5007_br_err'))
    hdus.append(fits.ImageHDU(output_par_err[3,:,:],name='amp_Hb_br_err'))
    hdus.append(fits.ImageHDU(output_par_err[4,:,:],name='amp_Hb1_err'))
    hdus.append(fits.ImageHDU(output_par_err[5,:,:],name='amp_Hb2_err'))
    hdus.append(fits.ImageHDU(output_par_err[6,:,:],name='amp_Fe5018_1_err'))
    hdus.append(fits.ImageHDU(output_par_err[7,:,:],name='amp_Fe5018_2_err'))
    hdus.append(fits.ImageHDU(output_par_err[8,:,:],name='m_err'))
    hdus.append(fits.ImageHDU(output_par_err[9,:,:],name='c_err'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s_MC.fits'%(destination_path_cube,obj,obj),overwrite='True')


In [5]:
######################## Scale Factor Error Flux Maps #####################
def err_ratio(data,err):
    data[2:-2,2:-2]=0
    err[2:-2,2:-2]=0
    edge_fluxmap = data[data!=0]
    edge_errmap = err[err!=0]
    err_fluxmap = np.std(edge_fluxmap)
    err_errmap = np.mean(edge_errmap)
    fact = err_fluxmap/err_errmap
    return fact

def flux_dat(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdu = fits.open('%s/Flux Maps/%s/9_arcsec_subcube_par_%s_MC.fits'%(destination_path_cube,obj,obj))
    Hb1 = hdu[5].data
    Hb2 = hdu[6].data
    OIII_br = hdu[3].data
    OIII_nr = hdu[2].data
    hdu.close()
    return Hb1,Hb2,OIII_br,OIII_nr

def flux_err(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdu = fits.open('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s_MC.fits'%(destination_path_cube,obj,obj))
    Hb1 = hdu[5].data
    Hb2 = hdu[6].data
    OIII_br = hdu[3].data
    OIII_nr = hdu[2].data
    hdu.close()
    return Hb1,Hb2,OIII_br,OIII_nr

In [6]:
def flux_data_err(obj,emp_Hb1,emp_Hb2,emp_wing,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdu = fits.open('%s/Flux Maps/%s/9_arcsec_subcube_par_%s_MC.fits'%(destination_path_cube,obj,obj))
    (OIII_nr,OIII_br,Hb1_blr_br,Hb2_blr_br) = (hdu[2].data,hdu[3].data,hdu[5].data,hdu[6].data)
    hdu.close()
    
    hdu = fits.open('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s_MC.fits'%(destination_path_cube,obj,obj))
    (OIII_nr_err,OIII_br_err,Hb1_blr_br_err,Hb2_blr_br_err) = (hdu[2].data,hdu[3].data,hdu[5].data,hdu[6].data)
    hdu.close()
       
    (amp_OIII_nr,amp_OIII_br,amp_Hb1_blr_br,amp_Hb2_blr_br) = (np.max(OIII_nr),np.max(OIII_br),np.max(Hb1_blr_br),np.max(Hb2_blr_br))
    if amp_Hb1_blr_br > amp_Hb2_blr_br:
        (Hb_blr_br,amp_Hb_blr_br,Hb_blr_err,emp_Hb_blr) = (Hb1_blr_br,amp_Hb1_blr_br,Hb1_blr_br_err,emp_Hb1)
    else:
        (Hb_blr_br,amp_Hb_blr_br,Hb_blr_err,emp_Hb_blr) = (Hb2_blr_br,amp_Hb2_blr_br,Hb2_blr_br_err,emp_Hb2)
    (blr_err_final,wing_err_final,core_err_final) = (emp_Hb_blr*Hb_blr_err,emp_wing*OIII_br_err,OIII_nr_err)
    return Hb_blr_br,OIII_br,OIII_nr,amp_Hb_blr_br,amp_OIII_br,amp_OIII_nr,blr_err_final,wing_err_final,core_err_final
    
def moffat_fit(data,error,box_size,amp,x0,y0,MC_loops,gamma_fixed=None,alpha_fixed=None):
    y, x = np.mgrid[:box_size, :box_size] 
    if gamma_fixed is None and alpha_fixed is None:
        p_init = models.Moffat2D(amplitude=amp,x_0=x0,y_0=y0,gamma=1,alpha=1)
    else:
        p_init = models.Moffat2D(amp,x0,y0,gamma_fixed,alpha_fixed,fixed={'gamma':True,'alpha':True})
    f = fitting.LevMarLSQFitter()
    p = f(p_init, x, y, data)
    model = p(x,y)
    residual = data - model
    res = (residual/error)
    [amp_out,x0_out,y0_out,gamma_out,alpha_out]= p.parameters
    fwhm_out = 2*gamma_out*np.sqrt(2**(1/alpha_out)-1)
    p_parameters = np.append(p.parameters,fwhm_out)
    parameters_MC = np.zeros((len(p_parameters),MC_loops))
    for l in range(MC_loops):
        iteration_data = np.random.normal(data,error) 
        if gamma_fixed is None and alpha_fixed is None:
            p_MC_init = models.Moffat2D(amplitude=amp,x_0=x0,y_0=y0,gamma=1,alpha=1)
        else:
            p_MC_init = models.Moffat2D(amp,x0,y0,gamma_fixed,alpha_fixed,fixed={'gamma':True,'alpha':True})
        f = fitting.LevMarLSQFitter()
        p_MC = f(p_MC_init, x, y, iteration_data)
        [amp_MC,x0_MC,y0_MC,gamma_MC,alpha_MC]= p_MC.parameters
        fwhm_MC = 2*gamma_MC*np.sqrt(2**(1/alpha_MC)-1)
        p_MC_parameters = np.append(p_MC.parameters, fwhm_MC)
        parameters_MC[:,l] = p_MC_parameters    
    parameters_err = np.std(parameters_MC,1) 
    [amp_err,x0_err,y0_err,gamma_err,alpha_err,fwhm_err] = parameters_err    
    if gamma_fixed is None and alpha_fixed is None:
        (par,err) = ([amp_out,x0_out,y0_out,gamma_out,alpha_out,fwhm_out],[amp_err,x0_err,y0_err,gamma_err,alpha_err,fwhm_err])   
    else:
        (par,err) = ([amp_out,x0_out,y0_out],[amp_err,x0_err,y0_err])   
    return par,err,model,res

In [7]:
############################### Extract all the 3" parameters###########################
def red_data_err(data,model,res,x0,y0,x_m,y_m,muse_sampling_size,box_size=4):
    if muse_sampling_size == 0.4:
        (y_cen,x_cen)=(y_m,x_m)
        data_red = data[y_cen-box_size:y_cen+box_size+1,x_cen-box_size:x_cen+box_size+1]
        mod_red = model[y_cen-box_size:y_cen+box_size+1,x_cen-box_size:x_cen+box_size+1]
        residual_red = res[y_cen-box_size:y_cen+box_size+1,x_cen-box_size:x_cen+box_size+1]
        (x0,y0)=(x0-7,y0-7)
    else:
        (y_cen,x_cen)=(y_m,x_m)
        data_red = data[y_cen-2*box_size:y_cen+2*box_size+1,x_cen-2*box_size:x_cen+2*box_size+1]
        mod_red =model[y_cen-2*box_size:y_cen+2*box_size+1,x_cen-2*box_size:x_cen+2*box_size+1]
        residual_red =res[y_cen-2*box_size:y_cen+2*box_size+1,x_cen-2*box_size:x_cen+2*box_size+1]
        (x0,y0)=(x0-14,y0-14)
    #plt.imshow(data_red,origin='lower')
    #splt.show()
    err_red = (data_red - mod_red)/residual_red
    return data_red,mod_red,residual_red,err_red,x0,y0 

In [8]:
####################### Extract the chi squared #########################################
def res_filter(res):
    res[res>10**10]=0
    return res

def chi_squared(data,model,err):
    res = data - model
    a = np.sum((res/err)**2)
    return a

def red_chi_squared(data,model,err,n_free):
    dof = len(data.flatten()) - n_free
    res = data - model
    k = res/err
    k[k>10**10]=0
    l = k[k!=0]
    a = np.sum(l**2)
    red = a/dof
    return red

In [9]:
def central_table(obj,parm_err,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    column_names={'red_chisq_blr':0,'red_chisq_wing':1,'red_chisq_core':2,'norm_chisq_wing':3,'norm_chisq_core':4,
                  'psf_amp_blr':5,'psf_amp_wing':6,'psf_amp_core':7,'flux_wing_data':8,'flux_wing_model':9,
                  'flux_ratio':10,'gamma':11,'alpha':12,'Hb_x':13,'Hb_y':14,'OIII_br_x':15,'OIII_br_y':16,
                  'OIII_nr_x':17,'OIII_nr_y':18,'off_wing_x':19,'off_wing_y':20,'off_core_x':21,'off_core_y':22}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key+'_err',format='E',array=[parm_err[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s/%s/%s_parameters_erractual.fits'%(destination_path_cube,obj,obj),overwrite=True)



In [10]:
def algorithm_script(obj,z,broad2,p_init,box_size=11,min_wave=4750,max_wave=5090,prefix_path_cube="/home/mainak/ftp.hidrive.strato.com/users/login-carsftp/IFU_data",destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    print ('%s'%(obj))
    (wo_data,wo_err,wo_wave,wo_header) = loadCube('%s/%s/%s.wo_absorption.fits'%(destination_path_cube,obj,obj)) 
    (cont_cube,cont_err,cont_wave,cont_header) = loadCube('%s/MUSE/%s/fitting/full/%s.cont_model.fits'%(prefix_path_cube,obj,obj))
    MC_loops = 100
    Monte_Carlo_loops = 30
    popt_MC = np.zeros((23,Monte_Carlo_loops))
    muse_sampling_size = sampling_size(cont_cube)
    [x0,y0] = agn_location(obj)
    [brightest_pixel_x,brightest_pixel_y] = [int(x0),int(y0)]
        
    for l in range(Monte_Carlo_loops):
        k=1+z
        wo_cube = np.random.normal(wo_data,wo_err)
        (data,error) = (wo_cube[:,brightest_pixel_y,brightest_pixel_x],wo_err[:,brightest_pixel_y,brightest_pixel_x])
        factor = scale_factor_spectrum(z,data,error,wo_wave)
        error = error*factor
        (popt_full_fit,parameters_err) = popt_central_fit(wo_wave,data,error,obj,p_init,z,MC_loops,broad2)
        [amp_Hb_fit,amp_OIII5007_fit,vel_OIII_fit,vel_sigma_OIII_fit,amp_Hb_br_fit,amp_OIII5007_br_fit,vel_OIII_br_fit,vel_sigma_OIII_br_fit,amp_Hb1_fit,amp_Fe5018_1_fit,vel_Hb1_fit,vel_sigma_Hb1_fit,amp_Hb2_fit,amp_Fe5018_2_fit,vel_Hb2_fit,vel_sigma_Hb2_fit,m_fit,c_fit] = popt_full_fit

        fixed_param = fixed_parameters(obj)
        (mini_cube,mini_err,wo_header)=create_9_arcsec_minicube(wo_cube,wo_err,wo_header,cont_cube,brightest_pixel_x,brightest_pixel_y,muse_sampling_size,box_size=11)
        true_err = emp_fact(z,mini_cube,mini_err,wo_wave)
        select = (wo_wave > min_wave*k) & (wo_wave < max_wave*k)  
        mini_header = wo_header
        mini_wave = wo_wave[select]
        mini_cube = mini_cube[select]
        mini_err = true_err[select]
        (output_par,output_par_err,fitted_cube,residual_cube) = spectroastrometry(mini_cube,mini_err,mini_wave,fixed_param,MC_loops,broad2)
        flux_map(output_par,obj)
        flux_map_err(output_par_err,obj)
        
        (Hb1,Hb2,wing,core) = flux_dat(obj)
        (Hb1_err,Hb2_err,wing_err,core_err) = flux_err(obj)
        (emp_Hb1,emp_Hb2,emp_wing) = (err_ratio(Hb1,Hb1_err),err_ratio(Hb2,Hb2_err),err_ratio(wing,wing_err))
        
        (Hb_blr_br_data,OIII_br_data,OIII_nr_data,amp_Hb_blr_br,amp_OIII_br,amp_OIII_nr,Hb_blr_br_err,OIII_br_err,OIII_nr_err) = flux_data_err(obj,emp_Hb1,emp_Hb2,emp_wing)
        box_length = np.shape(Hb_blr_br_data)[1]
        (brightest_pixel_Hb_blr_br_x,brightest_pixel_Hb_blr_br_y,brightest_pixel_OIII_br_x,brightest_pixel_OIII_br_y,brightest_pixel_OIII_nr_x,brightest_pixel_OIII_nr_y) = brightest_pixel_flux_map(Hb_blr_br_data,OIII_br_data,OIII_nr_data) 
        (Hb_par,Hb_error,Hb_model,blr_res) = moffat_fit(Hb_blr_br_data,Hb_blr_br_err,box_length,amp_Hb_blr_br,brightest_pixel_Hb_blr_br_x,brightest_pixel_Hb_blr_br_y,MC_loops,None,None)
        (gamma_fix,alpha_fix) = (Hb_par[3],Hb_par[4])#these two are gamma and alpha
        (OIII_br_par,OIII_br_error,OIII_br_model,wing_res) = moffat_fit(OIII_br_data,OIII_br_err,box_length,amp_OIII_br,brightest_pixel_OIII_br_x,brightest_pixel_OIII_br_y,MC_loops,gamma_fix,alpha_fix)   
        (OIII_nr_par,OIII_nr_error,OIII_nr_model,core_res) = moffat_fit(OIII_nr_data,OIII_nr_err,box_length,amp_OIII_nr,brightest_pixel_OIII_nr_x,brightest_pixel_OIII_nr_y,MC_loops,gamma_fix,alpha_fix) 
        
        (Hb_cen_x,Hb_cen_y) = (Hb_par[1],Hb_par[2])
        (OIII_br_cen_x,OIII_br_cen_y) = (OIII_br_par[1],OIII_br_par[2])
        (OIII_nr_cen_x,OIII_nr_cen_y) = (OIII_nr_par[1],OIII_nr_par[2])
        
        (Hb_amp,OIII_br_amp,OIII_nr_amp) = (Hb_par[0],OIII_br_par[0],OIII_nr_par[0])
        (y_m,x_m) = ndimage.measurements.maximum_position(Hb_blr_br_data)
        #print (x_m,y_m)
        Hb_res = res_filter(blr_res)
        OIII_br_res = res_filter(wing_res)
        OIII_nr_res = res_filter(core_res)
    
        (Hb_data,Hb_model,Hb_res,Hb_err,Hb_x,Hb_y) = red_data_err(Hb_blr_br_data,Hb_model,Hb_res,Hb_cen_x,Hb_cen_y,x_m,y_m,muse_sampling_size,box_size=4)
        (OIII_br_data,OIII_br_model,OIII_br_res,OIII_br_err,OIII_br_x,OIII_br_y) = red_data_err(OIII_br_data,OIII_br_model,OIII_br_res,OIII_br_cen_x,OIII_br_cen_y,x_m,y_m,muse_sampling_size,box_size=4)
        (OIII_nr_data,OIII_nr_model,OIII_nr_res,OIII_nr_err,OIII_nr_x,OIII_nr_y) = red_data_err(OIII_nr_data,OIII_nr_model,OIII_nr_res,OIII_nr_cen_x,OIII_nr_cen_y,x_m,y_m,muse_sampling_size,box_size=4)
        (y_m,x_m) = ndimage.measurements.maximum_position(Hb_data)
        #print (x_m,y_m)
        chi_squared_Hb = chi_squared(Hb_data,Hb_model,Hb_err)
        chi_squared_OIII_br = chi_squared(OIII_br_data,OIII_br_model,OIII_br_err)
        chi_squared_OIII_nr = chi_squared(OIII_nr_data,OIII_nr_model,OIII_nr_err)

        red_chi_squared_Hb = red_chi_squared(Hb_data,Hb_model,Hb_err,5)
        red_chi_squared_OIII_br = red_chi_squared(OIII_br_data,OIII_br_model,OIII_br_err,3)
        red_chi_squared_OIII_nr = red_chi_squared(OIII_nr_data,OIII_nr_model,OIII_nr_err,3)

        normalized_chi_squared_OIII_br = (red_chi_squared_OIII_br/red_chi_squared_Hb)
        normalized_chi_squared_OIII_nr = (red_chi_squared_OIII_nr/red_chi_squared_Hb)
    
        flux_wing_data = np.sum(OIII_br_data)
        flux_wing_model = np.sum(OIII_br_model)
        flux_ratio = (flux_wing_data/flux_wing_model)
        
        (offset_wing_x,offset_wing_y) = (OIII_br_x - Hb_x,OIII_br_y - Hb_y)
        (offset_core_x,offset_core_y) = (OIII_nr_x - Hb_x,OIII_nr_y - Hb_y)
        popt = [red_chi_squared_Hb,red_chi_squared_OIII_br,red_chi_squared_OIII_nr,normalized_chi_squared_OIII_br,normalized_chi_squared_OIII_nr,Hb_amp,OIII_br_amp,OIII_nr_amp,flux_wing_data,flux_wing_model,flux_ratio,gamma_fix,alpha_fix,Hb_x,Hb_y,OIII_br_x,OIII_br_y,OIII_nr_x,OIII_nr_y,offset_wing_x,offset_wing_y,offset_core_x,offset_core_y]
        print (popt)
        popt_MC[:,l] = popt
    parm_err = np.std(popt_MC,1)
    print ('norm chi-sq err and flux_err is', parm_err)
    #plt.imshow((OIII_br_data - OIII_br_model)/OIII_br_err,origin='lower')
    #plt.show()
    central_table(obj,parm_err)
    



In [12]:
z = {"HE0021-1810":0.05352}

objs = z.keys()

broad2= {'HE0021-1810':False}

p_init= {'HE0021-1810':[0.1,1.5248,16056,50.0,1.0,3.0,15956,100.0,1.0,1.0,16056,1000.0,0,0,16056,1000.0,-0.001,2.0]}

for obj in objs:
     algorithm_script(obj,z[obj],broad2[obj],p_init[obj]) 


HE0021-1810


/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide


[0.7555474723968514, 0.047348599080536444, 1.639850408947546, 0.06266793392919537, 2.1704134668671284, 0.17837944312240112, 1.4421242398895235, 2.2602293349743126, 11.236119, 11.025704113470129, 1.0190840561917052, 1.8276272917712677, 2.2836564555955987, 4.3297988218773895, 3.5873287377897416, 4.179632145338191, 3.7001162954359703, 4.219460062520914, 3.757893065320289, -0.15016667653919846, 0.11278755764622872, -0.11033875935647508, 0.1705643275305473]
[0.7182780031185577, 0.06365901969268184, 2.0602587684973335, 0.08862727163618067, 2.8683305900393425, 0.1551742884918875, 1.315644615959829, 2.077098100141295, 11.161934, 11.575506804489583, 0.9642717236878651, 2.128807629212317, 2.524217119387985, 4.333014254780206, 3.637835869125217, 4.174744242156665, 3.7110290961204218, 4.2137666702563585, 3.7509431666026156, -0.15827001262354123, 0.0731932269952047, -0.11924758452384765, 0.11310729747739856]
[0.8320837310925228, 0.04721562403780474, 1.821462550081202, 0.05674383751718209, 2.1890375

[0.6844430011161351, 0.05987802422518163, 1.9906041137715955, 0.08748431078634353, 2.9083562992469454, 0.1754540423437804, 1.445910924328261, 2.2806439149831834, 11.23204, 11.185996236577154, 1.0041162331653326, 1.7159131823028457, 2.0931279663712146, 4.318528725717927, 3.5596177220444893, 4.182491609888528, 3.7115660412700997, 4.2212599050029915, 3.736859428445868, -0.13603711582939937, 0.15194831922561036, -0.0972688207149357, 0.1772417064013787]
[0.6409123463594362, 0.04707802943190488, 1.64178726629722, 0.07345470827535378, 2.561640879011049, 0.15357942965648516, 1.3454034003945803, 2.1060147158957054, 11.206424, 11.476528254730379, 0.9764646163652672, 2.063643130100054, 2.4762400910624853, 4.339427502038292, 3.519334579951657, 4.189906204426066, 3.7048835790005192, 4.206997362673992, 3.7616649489868905, -0.14952129761222643, 0.18554899904886213, -0.13243013936430081, 0.24233036903523342]
[0.7423357278799277, 0.04566752079269031, 2.5006789130006792, 0.061518689021090735, 3.36866301

In [ ]:
z = {"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

broad2= {'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':False}

p_init= {'HE0021-1819':[1,12.5,15959,50.0,3,3,15759,200.0,2,2,15959,1000.0,0,0,15959,1000.0,-0.001,0.1]
        ,'HE0040-1105':[1.139,1.5,12507.0,50.0,1.0,3.0,12307.0,200.0,1.0,1.0,12507,1000.0,0,0,12507,1000.0,-0.001,2.0]
        ,'HE0108-4743':[1.7,4.5,7160.0,40.0,0.12,2.5,7100.0,120.0,1.0,1.0,6960,1300.0,1.0,1.0,7190,430.0,-0.001,2.0]
        ,'HE0114-0015':[0.1,1.5,13680,50.0,1.0,3.0,13480,100.0,1.0,1.0,13680,1000.0,0,0,13680,1000.0,-0.001,0.3]
        ,'HE0119-0118':[1,12.5,16302,50.0,3,3,16102,100.0,2,2,16302,1000.0,1,1,16302,2500.0,-0.001,0.1]
        ,'HE0212-0059':[0.1,1.0,7930,170.0,0.1,0.1,7630,450.0,1.0,1.0,7620,4000.0,0.1,0.1,4650,1090.0,-0.001,2.0]
        ,'HE0224-2834':[0.2,2.5,17940,50.0,0.3,0.3,17640,200.0,0.1,0.1,17940,1500.0,0,0,17940,1500.0,-0.001,0.1]
        ,'HE0227-0913':[3,6,4785,150,2,7,4385,300.0,7,1.8,4935,400.0,3,0.5,4735,1500.0,-0.3,4.7]
        ,'HE0232-0900':[2,40,12942,50.0,5,5,12742,100.0,5,1,12942,1000.0,0,0,12942,1000.0,-0.001,0.1]                                     
        ,'HE0253-1641':[2,10,9476,110.0,1,6,9176,370.0,1,1,9476,1500.0,0.5,0.5,9476,550.0,-0.001,0.5]
        ,'HE0345+0056':[0.5,20,9300,150,5,10,9000,300,9,1.8,9300,400,30,2.5,9300,1500,0.3,7.1]
        ,'HE0351+0240':[2.2,37,10600,77,1.1,1.5,10618,277,1.5,0.07,11580,320,3,0.35,10650,1344,-0.3,3]                                                    
        ,'HE0412-0803':[1.139,1.5,11448,50.0,0.1,0.3,11248,100.0,0.1,0.1,11448,1000.0,0,0,11448,1000.0,-0.001,0.5]
        ,'HE0429-0247':[0.9,8.5,12600,90,0.2,2.0,12500,220,1.7,0.07,12600,450,1.3,0.2,12600,1600,-0.5,4.0]
        ,'HE0433-1028':[0.9,4.0,10665.0,100.0,1.0,1.0,10465.0,300.0,1.0,1.0,10665,800.0,0.1,0.1,10665,1500.0,-0.1,1.0]
        ,'HE0853+0102':[0.1,0.9,15700,110.0,0.02,0.3,15670,320.0,0.1,0.1,15900,1640.0,0.0,0.0,17250,810.0,-0.001,0.1]
        ,'HE0934+0119':[0.6,1.9,15101,100.0,0.2,0.2,14901,400.0,0.3,0.1,15101,1000.0,0.1,0.1,15101,1000.0,0.001,0.7]
        ,'HE1011-0403':[0.1,0.6,17494,60.0,0.2,0.2,17244,200.0,0.3,0.1,17494,1500.0,0.1,0.1,17494,1500.0,0.001,0.7]
        ,'HE1017-0305':[0.1,1.5,14695,50,0.08,0.4,14595,200,0.7,0.09,14695,2000,0,0,0,0,-0.3,2.0]
        ,'HE1029-1831':[0.6,1.3,12078,50.0,0.2,0.2,11978,200.0,0.3,0.1,12078,1000.0,0.2,0.2,12078,1000.0,-0.001,0.2]
        ,'HE1107-0813':[0.04,0.3,17500,100,0.2,0.13,17400,480,0.9,0.13,17500,1000,0.9,0.08,16800,2000,-0.9,8]
        ,'HE1108-2813':[1.139,1.5,7200.0,50.0,1.0,3.0,7000.0,100.0,1.0,1.0,7200,1000.0,0,0,7200,1000.0,-0.001,2.0]
        ,'HE1126-0407':[1,13,18200,100,2.5,3,18000,300,10,1.3,18200,800,7,1.8,18200,1600,-0.9,15]
        ,'HE1237-0504':[1,10,2700.0,80.0,0.5,2,2700.0,200,2,0.5,2700.0,1500.0,0,0,2700.0,1000.0,-0.001,0.002]
        ,'HE1248-1356':[0.1,1.5,4395.0,50.0,1.0,3.0,4195.0,100.0,1.0,1.0,4395,1000.0,0,0,0,0.0,-0.001,2.0]
        ,'HE1330-1013':[0.15,0.6,6643,90,0.05,0.15,6543,200,0.12,0.04,6643,1500,0.3,0.04,6643,500,-0.02,0.3]
        ,'HE1353-1917':[0.07,0.7,10490.0,80.0,0.39,0.05,10306.0,490.0,0.1,0.02,8600.0,1500.0,0.14,0.02,12326.0,1500.0,-0.001,0.002]
        ,'HE1417-0909':[1,12.5,13200,50.0,3,3,13000,100.0,2,2,13200,1000.0,0,0,13200,1000.0,-0.001,0.1]
        ,'HE2128-0221':[0.1,1.5,15744,50.0,1.0,3.0,15544,100.0,1.0,1.0,15744,1000.0,0,0,15744,1000.0,-0.001,2.0]
        ,'HE2211-3903':[0.6,2.4,11914,50.0,0.1,0.1,11714,100.0,0.1,0.1,11914,200.0,0,0,11914,100.0,-0.001,0.2]
        ,'HE2222-0026':[0.08,0.4,17400.0,140.0,0.04,0.05,17150.0,300.0,0.4,0.1,18500,650.0,0.02,0.01,17460,1750.0,-0.001,0.002]
        ,'HE2233+0124':[0.1,1.2,16944.0,100.0,1.0,3.0,17044.0,300.0,1.0,1.0,16944,1200.0,1.0,1.0,16944,4000.0,-0.001,2.0]
        ,'HE2302-0857':[1,8,14058,200,0.1,2,14258,300,1,0.1,14058,1000,0,0,0,0,0.01,0.2]}

for obj in objs:
     algorithm_script(obj,z[obj],broad2[obj],p_init[obj]) 


HE0021-1819


[1.3438925405478515, 3.0985494905055533, 33.597416246759565, 2.3056527192586382, 25.000076444403238, 0.05878935978954907, 0.34995449580348653, 1.8169133474583667, 8.572663, 8.353150204556918, 1.0262790800186103, 3.3220354595437676, 2.381918125496154, 7.615628582727833, 8.083930986384125, 7.361156017537262, 8.270748515591087, 7.487377790133394, 8.033973721412252, -0.2544725651905715, 0.18681752920696226, -0.1282507925944394, -0.04995726497187292]
[1.2143828003071007, 2.9918541130785408, 35.52317107601569, 2.463682878514042, 29.25203738642574, 0.05923187277039744, 0.35186419354861687, 1.8288603390194134, 8.57328, 8.31626997830648, 1.0309045229215266, 3.28450451617099, 2.3633785753521632, 7.63275301215052, 8.100863189756996, 7.363451621998404, 8.270711403792685, 7.487184064242825, 8.039760627920412, -0.2693013901521155, 0.16984821403568873, -0.14556894790769448, -0.06110256183658436]
[1.2871778627438992, 3.0031513090777007, 36.54091590655347, 2.333128463440034, 28.3883967897479, 0.0595815

[1.1323829777977732, 3.5449492166417222, 38.01301661860889, 3.130521463273707, 33.569046306697, 0.0602084143442634, 0.3544172585970395, 1.841879310545538, 8.569571, 8.28759212253673, 1.0340241670530954, 3.2479739621971966, 2.346903054543954, 7.615614077973891, 8.105895770185182, 7.371722896435742, 8.278297157611693, 7.483996303461669, 8.037061661167979, -0.24389118153814948, 0.17240138742651112, -0.13161777451222179, -0.0688341090172031]
[1.1388661071350854, 3.1007592575305254, 35.79067119826653, 2.7226723476131443, 31.426583839869473, 0.058978740598133156, 0.35063325243505994, 1.8270542841773258, 8.578018, 8.322330097151763, 1.0307231374314636, 3.292728517473363, 2.364142522339099, 7.6284176731180295, 8.095199458416939, 7.367603661486054, 8.25785944756948, 7.488310876619728, 8.049949956660235, -0.26081401163197526, 0.16265998915254087, -0.14010679649830138, -0.045249501756703836]
[1.1870704120537272, 3.0480662965689684, 32.89556342102956, 2.567721565307629, 27.711551974509742, 0.05930

[2.291398130013618, 2.5933002631428526, 460.39138883258823, 1.1317545515878729, 200.92160450085208, 0.3790499189006334, 0.6351758373760029, 3.2664140047670003, 13.305843, 13.062875838525224, 1.0185998487430843, 3.1630617881565555, 2.4756483808872782, 7.642395748490948, 7.813920088355186, 8.152834282664784, 7.677456185674163, 7.5167680322893204, 7.895448961094846, 0.5104385341738364, -0.13646390268102238, -0.1256277162016275, 0.08152887273966059]
[2.310182109297301, 2.6553583168787687, 441.751829958173, 1.1494151505166237, 191.2194835984349, 0.3787219287748382, 0.6353445120461342, 3.271571152080273, 13.338015, 13.048761289978493, 1.0221671089120763, 3.1364137115631148, 2.451081752125445, 7.645850017915404, 7.815846484959145, 8.150294999245368, 7.683993297374471, 7.518138109577247, 7.8815496487374865, 0.5044449813299643, -0.13185318758467446, -0.12771190833815638, 0.06570316377834118]
[2.072881377890894, 2.549177014474357, 447.63768864087814, 1.2297746709790418, 215.94949591198437, 0.375

[2.2357783617338196, 2.497279639346796, 437.90834494861116, 1.116962075529788, 195.8639337617609, 0.37931607530205685, 0.6378447250091459, 3.2740206161905654, 13.327417, 13.079139377901047, 1.0189827471504493, 3.134740357691193, 2.452058985268965, 7.639957074902103, 7.810315322513201, 8.140662711020639, 7.686065752726179, 7.518896803721766, 7.894806680267049, 0.5007056361185356, -0.12424956978702184, -0.1210602711803368, 0.08449135775384775]
[2.26902827104324, 2.3606547093888754, 441.0415969583852, 1.0403813559817428, 194.37465922608615, 0.37802135475995824, 0.6359400957457423, 3.254452037679103, 13.373228, 13.076668532731162, 1.022678523941068, 3.1888246860197813, 2.502013466247785, 7.63730255025899, 7.812671030680722, 8.133533705796378, 7.6860559841488225, 7.522746092081707, 7.892449118720926, 0.4962311555373873, -0.12661504653189937, -0.1145564581772831, 0.0797780880402037]
[1.9446653455092624, 2.3942476219557642, 450.9600718436048, 1.231187477827331, 231.89597782723325, 0.378016870

...

[20.201405357268975, 9.70324996927442, 5387.589171131576, 0.48032549209666475, 266.69378074694123, 1.6171035528366817, 4.914805593585649, 31.00414704426251, 129.7622, 121.64772580453868, 1.066704750730396, 3.4029530924245908, 2.3975739236703038, 8.029931453325094, 7.994797613821124, 8.124608186965169, 8.035126542745452, 8.20016997859403, 8.257475387016726, 0.09467673364007467, 0.04032892892432827, 0.17023852526893535, 0.2626777731956018]
[21.43457223882522, 10.329886433474814, 5408.410996249454, 0.4819264092783674, 252.3218534985738, 1.6051216244760387, 4.905719907727622, 30.963287209752338, 129.87164, 121.76680382637073, 1.0665603348806982, 3.437066984069023, 2.4239775056863966, 8.026065647926721, 7.9951655176524845, 8.136772011590502, 8.024250451086356, 8.192793245153844, 8.270678599493234, 0.1107063636637804, 0.02908493343387164, 0.16672759722712271, 0.2755130818407494]
[21.443913254383748, 9.33568423006551, 5488.6537040512785, 0.43535357186529516, 255.95392216619982, 1.606849485483

[22.647450034873206, 9.005688108473993, 5302.466283909045, 0.3976468915752887, 234.1308304353979, 1.61390836553212, 4.938705466754722, 30.899926137558992, 130.36865, 122.65995141800673, 1.0628461110299414, 3.415319016286648, 2.403126975374788, 8.02770630957285, 7.992775971647099, 8.136370705471936, 8.024164227066073, 8.194922698689776, 8.264313892220933, 0.10866439589908694, 0.03138825541897461, 0.16721638911692693, 0.2715379205738344]
[22.002578091679528, 9.24084597681623, 5251.8767834771525, 0.41998923663908, 238.69370041973386, 1.6128930677420943, 4.921700537989854, 30.969333105492012, 130.10823, 122.27984700009921, 1.0640202272310233, 3.4132601404607326, 2.4006823168017752, 8.025986420509387, 7.994284455009833, 8.136512728265714, 8.023457670883854, 8.192050651790858, 8.264377844077686, 0.11052630775632721, 0.029173215874020997, 0.16606423128147085, 0.27009338906785274]
norm chi-sq err and flux_err is [1.43290571e+00 4.09515750e-01 7.68484410e+01 3.56638408e-02
 1.90288058e+01 3.786

[3.2684378838761288, 7.760744796152958, 272.54647387379777, 2.374450753504693, 83.38738062556585, 1.7938172189460322, 2.185989178867407, 9.292662741410757, 22.760904, 23.908979864066918, 0.9519814078868926, 2.460906740491114, 2.61496315850434, 4.034610574921709, 3.8197913500573186, 4.123246723289675, 3.8058328697064425, 3.9357674122330852, 3.842056310541958, 0.08863614836796607, -0.013958480350876101, -0.09884316268862392, 0.022264960484639218]
[3.67663502531156, 7.460029110952598, 273.6899414200739, 2.029037165667656, 74.44033458199493, 1.7700704665069664, 2.160146088765269, 9.25276399689703, 22.773186, 23.876597319340817, 0.9537868995903135, 2.5153783739367563, 2.674325592684378, 4.042110705024037, 3.823016026422666, 4.125959683133651, 3.809186313205501, 3.9336622443264595, 3.842660886986767, 0.08384897810961434, -0.013829713217164752, -0.10844846069757708, 0.019644860564101307]
[4.235653745117597, 6.988321228263349, 262.3698873485873, 1.6498801953107542, 61.943185901590496, 1.789703

[2.281646331910265, 4.877993357982583, 8.871886132320538, 2.1379270265337644, 3.888370431578991, 2.5321894406104986, 5.0484168911349485, 4.776281723289673, 88.50048, 88.447702238559, 1.0005967188740994, 2.8870220958267794, 2.451951609220628, 7.815012926195113, 8.108279060622408, 7.878762279088772, 8.037188967009289, 7.878649598997008, 8.227503472413005, 0.0637493528936588, -0.07109009361311891, 0.06363667280189489, 0.11922441179059717]
[1.9341343603306074, 4.541759045056155, 7.494895556376563, 2.3482127913180855, 3.875064581912209, 2.5091527893322816, 5.030576002685031, 4.767961703761426, 88.53778, 88.7411019109941, 0.9977088277596634, 2.883438490460526, 2.4370437295937997, 7.8131370140529945, 8.110053368358468, 7.8797558954312095, 8.034527872823638, 7.876474385144963, 8.234628996093896, 0.06661888137821492, -0.0755254955348299, 0.06333737109196846, 0.12457562773542818]
[2.212091127923589, 4.777319848766456, 9.219229557540446, 2.1596397130577323, 4.167653602134469, 2.52075349629214, 5.

[2.294053486717288, 4.718341414463571, 9.06948740867871, 2.0567704466277967, 3.9534768745330497, 2.496846844055837, 5.03179256176681, 4.749531349639601, 88.57959, 88.84243130508682, 0.9970414872997542, 2.9081572796241146, 2.4621412994509826, 7.818994019958261, 8.116430751400124, 7.87832174495103, 8.033461262543579, 7.8760940198833325, 8.230680860893404, 0.059327724992769504, -0.08296948885654487, 0.05709999992507164, 0.1142501094932804]
[2.3110029332225084, 4.543607121011233, 9.093247692438272, 1.9660758780065835, 3.934762505800227, 2.5203049654658507, 5.053065840627545, 4.771984286472239, 88.62335, 88.63729142565737, 0.9998427369039383, 2.890530815048564, 2.4537571052791614, 7.811947814080181, 8.112323133222937, 7.878046017853922, 8.035614154362268, 7.883403105886607, 8.229989801902747, 0.0660982037737412, -0.07670897886066896, 0.0714552918064264, 0.11766666867981002]
[2.336474384985064, 4.8991196460594, 9.36305464034621, 2.096800066605788, 4.007343157928976, 2.519985175699608, 5.0560

In [11]:
z = {"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

broad2= {'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':False}

p_init= {'HE1353-1917':[0.07,0.7,10490.0,80.0,0.39,0.05,10306.0,490.0,0.1,0.02,8600.0,1500.0,0.14,0.02,12326.0,1500.0,-0.001,0.002]
        ,'HE1417-0909':[1,12.5,13200,50.0,3,3,13000,100.0,2,2,13200,1000.0,0,0,13200,1000.0,-0.001,0.1]
        ,'HE2128-0221':[0.1,1.5,15744,50.0,1.0,3.0,15544,100.0,1.0,1.0,15744,1000.0,0,0,15744,1000.0,-0.001,2.0]
        ,'HE2211-3903':[0.6,2.4,11914,50.0,0.1,0.1,11714,100.0,0.1,0.1,11914,200.0,0,0,11914,100.0,-0.001,0.2]
        ,'HE2222-0026':[0.08,0.4,17400.0,140.0,0.04,0.05,17150.0,300.0,0.4,0.1,18500,650.0,0.02,0.01,17460,1750.0,-0.001,0.002]
        ,'HE2233+0124':[0.1,1.2,16944.0,100.0,1.0,3.0,17044.0,300.0,1.0,1.0,16944,1200.0,1.0,1.0,16944,4000.0,-0.001,2.0]
        ,'HE2302-0857':[1,8,14058,200,0.1,2,14258,300,1,0.1,14058,1000,0,0,0,0,0.01,0.2]}

for obj in objs:
     algorithm_script(obj,z[obj],broad2[obj],p_init[obj]) 


HE1353-1917


[1.1032728329492707, 14.372295343212773, 311.796609548786, 13.026963878728646, 282.61061111718925, 0.13197347157878875, 0.17045070811240895, 1.2462452177129497, 6.369217, 3.936804418974417, 1.6178647047456238, 3.2759980388788006, 2.3184661209883757, 8.439861430277393, 7.5238316702468175, 10.716263044995781, 4.129052142713956, 8.37321481103427, 8.730188839617295, 2.276401614718388, -3.3947795275328616, -0.06664661924312298, 1.2063571693704773]
[1.147493630048786, 14.849839551039816, 310.4916548117877, 12.941108483895002, 270.582464844347, 0.1299114013925592, 0.16990782447551775, 1.2283558270760249, 6.400602, 3.9962634180172274, 1.6016466369568763, 3.3926718643930456, 2.401383888787731, 8.421099771950324, 7.533934258712804, 10.62553172664876, 4.241023403035971, 8.371376530108147, 8.721972999721299, 2.2044319546984354, -3.2929108556768334, -0.04972324184217669, 1.1880387410084943]
[1.2009640928036394, 16.57916233861098, 308.1480985778086, 13.804877629527693, 256.58393987320613, 0.13208329

[1.0687760854788562, 15.59244993071984, 299.94850544673, 14.589070753518753, 280.64672247259404, 0.13299203406259613, 0.17133122068427784, 1.2357200713527705, 6.423044, 3.981042154345577, 1.613407734881864, 3.3552117838311823, 2.391568076763994, 8.445996751977464, 7.504557974406822, 10.546285431501726, 4.303972050670197, 8.341446306608887, 8.7756226919598, 2.1002886795242617, -3.200585923736625, -0.10455044536857727, 1.271064717552978]
[0.9818151243457321, 16.32912837979797, 308.5398225666711, 16.631571438339243, 314.2545016020992, 0.13099482280762748, 0.17151468521708338, 1.2353930854186672, 6.4441805, 4.037372695537373, 1.5961321816312297, 3.3402832155377733, 2.355142344858484, 8.45349398125882, 7.51946015304733, 10.577892704042256, 4.317975901228369, 8.365962344295582, 8.74325722532831, 2.124398722783436, -3.201484251818961, -0.08753163696323796, 1.2237970722809806]
[1.2014151229527121, 16.358752705220514, 306.9257136744593, 13.61623671343148, 255.47015998943766, 0.13091084950592702

[6.326332653936065, 5.476340779530562, 19.495158168649482, 0.865642241579462, 3.0815891662794788, 1.6035961189938495, 2.7947680245126034, 14.424904933643532, 20.256353, 19.34659812881554, 1.0470240423366906, 1.930904092141413, 2.6305316596402184, 4.224918227347988, 4.1619334248838005, 4.323143795737964, 4.130801934296896, 4.24407392307544, 4.168659565920173, 0.0982255683899762, -0.031131490586904675, 0.019155695727452127, 0.006726141036372368]
[6.687532263624278, 6.818532723172618, 23.321359630165514, 1.0195887592589117, 3.4872892886092197, 1.5826005027441619, 2.7649739710189194, 14.283294912589712, 20.17718, 19.164563580424335, 1.0528379241131196, 2.033828118847558, 2.820112720719978, 4.218597930711555, 4.160935772815199, 4.320995340082572, 4.13211910325866, 4.241798797942371, 4.169075681515341, 0.10239740937101693, -0.028816669556539054, 0.023200867230816158, 0.008139908700142584]
[6.723661069461279, 5.443727183025401, 19.943016240975606, 0.8096373578006022, 2.966094815747978, 1.6004

[7.464027355367034, 6.00794086391279, 22.187682312073523, 0.8049194594112467, 2.9726153530398567, 1.5893885052214838, 2.7735627388629602, 14.301411969780872, 20.195581, 19.181510536690897, 1.052867103324662, 2.0278075627591114, 2.8131485093494613, 4.223411083909408, 4.160974480005008, 4.324278234513843, 4.1228442956662565, 4.239630341231463, 4.17225023418165, 0.10086715060443474, -0.03813018433875115, 0.01621925732205476, 0.011275754176642394]
[6.666183072063237, 5.883438752834583, 21.986584961303198, 0.8825798345519502, 3.2982269949118233, 1.603515480413128, 2.791819828980219, 14.41706719574985, 20.21654, 19.317492287485162, 1.046540569659337, 1.9335152368599868, 2.6361696682891322, 4.2245312758563305, 4.162208174946334, 4.32080638852819, 4.127862487812216, 4.244188502149083, 4.175425539591165, 0.09627511267185973, -0.03434568713411856, 0.019657226292752128, 0.013217364644830809]
[6.964107754079049, 7.237427153736433, 21.833178471621807, 1.0392468654002796, 3.1351006105317047, 1.58352

...

[1.337502504963832, 0.7654666871840491, 2.2499364971809896, 0.5723104699566514, 1.682192361383152, 0.1936720721606023, 0.08679269602481665, 0.45482585583949503, 1.6726267, 1.6759232305675362, 0.9980330263776386, 2.725053161753195, 2.1434808932370046, 7.848078917884237, 7.8755234332650765, 8.02289069086303, 7.882673597929127, 7.851443465545742, 7.92120751438496, 0.17481177297879213, 0.007150164664050607, 0.0033645476615049574, 0.04568408111988376]
[1.5054572981443624, 0.6847796021411937, 2.2441958073858888, 0.4548648460406403, 1.4907070497131343, 0.19575861482309168, 0.08857659092812616, 0.4564740726910697, 1.6905885, 1.7020086686993983, 0.9932901667096423, 2.703734985830942, 2.130047925620389, 7.8776820103358, 7.874658523887817, 8.021821366980259, 7.9111481965119665, 7.84390659453501, 7.910346028686945, 0.14413935664445887, 0.036489672624149705, -0.03377541580078969, 0.03568750479912808]
[1.3302654984290756, 0.796479766499976, 2.099089452489993, 0.5987374456005566, 1.5779477517599527, 

[1.314118125861577, 0.8704805417517061, 2.298357128794387, 0.6624066167423052, 1.7489730059750237, 0.19601776606170557, 0.08822229502188474, 0.4574492150569341, 1.6848366, 1.6962287985150084, 0.9932838232247174, 2.640957277880252, 2.070605291080536, 7.85665457314839, 7.887034023663407, 8.029030264063739, 7.9218777317326, 7.86591298488246, 7.916398948791919, 0.17237569091534866, 0.03484370806919301, 0.009258411734069938, 0.029364925128511743]
[1.238634204742947, 0.75466389074953, 2.50738805816433, 0.6092709920812698, 2.0243168229676707, 0.19623874762468813, 0.08923061102412132, 0.4549533271662397, 1.689164, 1.7087796540607005, 0.9885206898728884, 2.6725515684996535, 2.1053028873203914, 7.865074596998809, 7.873291916207254, 8.025587567978363, 7.9649760037651625, 7.851694161991034, 7.891770562246144, 0.16051297097955342, 0.09168408755790836, -0.013380435007775304, 0.018478646038889934]
norm chi-sq err and flux_err is [0.13715551 0.06112536 0.14570817 0.08843435 0.2641736  0.00147359
 0.00

[1.1404155134760006, 1.0109574154484273, 8.955540353962999, 0.8864816406846454, 7.852874893525782, 0.21722258786160623, 0.14181966226279932, 0.9392969455086756, 1.1498266, 1.2524913490418357, 0.9180316069494295, 2.4862046764679446, 3.1283555240949106, 3.965452858845472, 3.99001900859661, 3.9666869235545708, 3.9961028382356254, 3.897542786799292, 3.972164961770922, 0.0012340647090987034, 0.006083829639015548, -0.06791007204618005, -0.01785404682568803]
[1.0552382436781929, 1.1664459950575077, 9.992329585885232, 1.1053863921683538, 9.46926406974737, 0.2172263354438952, 0.14278302777237897, 0.9376113939953399, 1.1290759, 1.2556081796141172, 0.899226289817598, 2.5201712791765187, 3.2011065015225784, 3.958255628654234, 3.988232017388338, 3.936106387610611, 3.9965576441408377, 3.9045310871022654, 3.9684830860918066, -0.022149241043623036, 0.008325626752499815, -0.053724541551968485, -0.019748931296531325]
[0.9705726720902307, 0.9770171288064601, 9.307594067609468, 1.0066398497521578, 9.58979

[1.5300776069645647, 3.729062385872258, 235.8620736927722, 2.4371720551287175, 154.15039905112118, 1.3030850262339213, 1.2617289776215734, 10.320391673006252, 26.42823, 27.95776772344549, 0.945291145812107, 3.3312803512842675, 2.5167626183961995, 7.976596396268086, 7.8523906871879525, 8.11211730599257, 7.952327779850698, 7.374975490141193, 7.66065722490767, 0.13552090972448383, 0.09993709266274564, -0.6016209061268931, -0.19173346228028265]
[1.5088635414822673, 3.5293381477980486, 243.37386030450324, 2.339070466459095, 161.29613686961994, 1.3085217612528797, 1.2687648293920557, 10.292165589722783, 26.628868, 28.05562863087697, 0.9491453017638183, 3.3547659179021903, 2.5436167660105022, 7.975556420763102, 7.857954354697068, 8.111903572685726, 7.952767063824449, 7.376095335092803, 7.656436820312674, 0.13634715192262448, 0.09481270912738182, -0.5994610856702991, -0.20151753438439357]
[1.4532497310852204, 3.6922228266804145, 237.82225787777026, 2.540666444110216, 163.6485820645399, 1.30675

[1.6600382736603287, 3.3302250278738748, 240.06779822225522, 2.006113401548773, 144.6158212321899, 1.3092642301571058, 1.2676233605982024, 10.350535953124796, 26.444744, 27.943095861893127, 0.9463784628878916, 3.302094273889072, 2.4969271181261563, 7.983135051662497, 7.86131712990737, 8.111838086843346, 7.924072538321795, 7.374042500245615, 7.66333488803301, 0.12870303518084825, 0.06275540841442506, -0.6090925514168823, -0.1979822418743602]
[1.66625819738363, 3.501110889521028, 218.62563948644618, 2.1011814945717875, 131.20754024180266, 1.3139313320088484, 1.263976650155783, 10.367364520763294, 26.462177, 27.923302712010262, 0.9476736168902227, 3.280792012072906, 2.4723594774154645, 7.977535969201206, 7.857541405351149, 8.12558953983282, 7.951768221208482, 7.375742555555078, 7.656930987620161, 0.14805357063161395, 0.09422681585733272, -0.6017934136461278, -0.20061041773098864]
[1.308437492802261, 3.2958572925540364, 239.62935248914326, 2.5189260554551582, 183.14161265429092, 1.29975617